<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Experiments" data-toc-modified-id="Experiments-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Experiments</a></span></li><li><span><a href="#Deliverables" data-toc-modified-id="Deliverables-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Deliverables</a></span></li></ul></li><li><span><a href="#Code" data-toc-modified-id="Code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Replace-with-Mode" data-toc-modified-id="Replace-with-Mode-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Replace with Mode</a></span></li></ul></li><li><span><a href="#Remove-'-?'-in-workclass,-occupation,-native-country" data-toc-modified-id="Remove-'-?'-in-workclass,-occupation,-native-country-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Remove ' ?' in workclass, occupation, native-country</a></span></li><li><span><a href="#Convert-Discrete-to-OneHot" data-toc-modified-id="Convert-Discrete-to-OneHot-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Convert Discrete to OneHot</a></span></li><li><span><a href="#KNN-Classifiers" data-toc-modified-id="KNN-Classifiers-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>KNN Classifiers</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li></ul></li></ul></div>

## Preprocessing
Your first task is to acquire the data, analyze it, and clean it (if necessary).  You will use two datasets in this project,outlined below.
• Dataset 1 (Adult dataset): This dataset presents several attributes of different individuals and the predictiontask is to determine whether someone makes over 50K a year.  Download and read information about the datasethere.

The essential subtasks for this part of the project include:

1.  Download the datasets.Hints: For clarity, in the Adult dataset, adult.data contains the training/validation dataand adult.test contains the test data.
2.  Load the datasets into Pandas dataframes or NumPy objects (i.e., arrays or matrices) in Python.
3.  Clean the data.  You should remove instances that have too many missing or invalid data entries.
4.  Convert discrete variables into multiple variables using one-hot encoding.  For an example on how to do this,check out ”Encoding categorical features” in the scikit-learn documentation.

## Experiments
In this part, you will compare two supervised learning frameworks, namelyK-nearest neighbours(KNN) anddecisiontrees,  to predict whether the income of an adult exceeds $50K/yr.  A similar analysis should be performed for thesecond dataset.  The specific subtasks for this part include:
1.  Implement and perform 5-fold cross validation on the training/validation data (for theAdultdataset, this datais contained in theadult.datafile) to optimize hyperparameters for both models.Your implementation forcross-validation should be from scratch.  You should not use existing packages for cross validation.Report the mean of the training and validation metrics for the given hyperparameters.
2.  Sample growing subsets of the training/validation data and repeat step 1.  We want to understand how the sizeof a dataset impacts both the training and validation error.
3.  Take the best performing model (the one with the best performance on 5-fold cross validation) and apply it onthe test set (in theAdultdataset, this is theadult.testfile).  This is an unbiased estimate of how your modelwould perform on new/unseen data.
4.  [Optional] Go above and beyond!  Examples:  different normalization techniques or other ways of handling of missing data (search “data imputation” techniques).  Employ more sophisticated techniques for hyper-parameter search.  Engineering new features out of existing ones to get a better performance.  Investigate which featuresare the most useful (e.g., by correlating them with your predictions or removing them from your data)?
5.  Analyze your findings; how did the choice of the various hyper-parameters impact generalization?  How aboutthe size of training data?  If any of these findings do not agree with your expectation, you can form hypothesesand further investigate them.

## Deliverables
You must submit two separate files to MyCourses(using the exact filenames and file types outlined below):

1. code.zip:  Your  entire  code,  which  should  consist  of  a  jupyter  notebook  file  (.ipynb),  and  additional  pythonfiles (.py);the notebook should contain the main body of your code, where we can see and easily reproduce the plots in your report.
2. writeup.pdf:  Your (max three pages) project write-up as a pdf (details below).

# Code

1. Download the datasets.Hints: For clarity, in the Adult dataset, adult.data contains the training/validation data and adult.test contains the test data.

In [1]:
from utils import *

# Vars
data_rootdir = 'Adult dataset'
remove_dup = True

In [2]:
# TODO Put in .py
import os
import pandas as pd
import numpy as np

rootdir = 'Adult dataset'

adult_names = ["age",
"workclass",
"fnlwgt",
"education",
"education-num",
"marital-status",
"occupation",
"relationship",
"race",
"sex",
"capital-gain",
"capital-loss",
"hours-per-week",
"native-country"]

def load_adult(rootdir=rootdir, split='train'):
    csv_name = 'adult.data' if split == 'train' else 'adult.test'
    csv_path = os.path.join(rootdir, csv_name)
    assert os.path.exists(csv_path)
    adult_data = pd.read_csv(csv_path, names=adult_names + ['salary'], index_col=False)
    return adult_data

def get_faulty_col_names(df):
    faulty_cols = []
    
#     Check NA values
    for col_name in df.columns.tolist():
        if df[col_name].isna().values.any() is True:
            faulty_cols.append(col_name)

#     Check for ' ?'
        if ' ?' in df[col_name].unique().tolist():
            if col_name not in faulty_cols:
                faulty_cols.append(col_name)
            print(f"Number of ' ?' in {col_name}: {df[col_name].value_counts()[' ?']}")
    
    return faulty_cols

def replace_with_mode(df, faulty_cols):
    
    df_without_QM = df.copy()
    for col in faulty_cols:
        df_without_QM = df_without_QM[getattr(df, col) != ' ?']
    
    r_df = df.copy()
    mapping = {}
    for col in faulty_cols:
        mapping[col] = df_without_QM[col].mode().values[0]
        r_df[col] = r_df[col].replace(' ?', mapping[col])
    return r_df, mapping

def replace_with_mapping(df, faulty_cols, mapping):
    r_df = df.copy()
    for col in faulty_cols:
        r_df[col] = r_df[col].replace(' ?', mapping[col])
    return r_df

def get_binary_labels_for_salary(df):
    if isinstance(df, pd.Series):
        df = df.to_frame()
    assert isinstance(df, pd.DataFrame)
    salary_map = {' <=50K': 0, ' <=50K.': 0, ' >50K': 1, ' >50K.': 1}
    df['salary'] = df['salary'].map(salary_map)
    
    return df['salary'].tolist()

def generate_cross_validation(n_folds, X, y):
    samples = len(X)
    elements_in_fold = samples // n_folds
    indices = [0]
    while indices[-1] + elements_in_fold <= samples - samples % n_folds:
        indices.append(indices[-1] + elements_in_fold)
    indices[-1] = samples
    cross_val_sets = [[indices[i], indices[i+1]] for i in range(len(indices)-1)]
    for val_set in cross_val_sets:
        begin, end = val_set[0], val_set[1]
        train_X_fold = X[0:begin].append(X[end:])
        validation_X_fold = X[begin:end]
        train_y_fold = y[0:begin] + y[end:]
        validation_y_fold = y[begin:end]
        yield train_X_fold, train_y_fold, validation_X_fold, validation_y_fold

def knn_fit(X_train, y_train, X_val, y_val, one_hot, knn=3):
    neigh = KNeighborsClassifier(n_neighbors=knn)
    neigh.fit(one_hot.transform(X_train), y_train)
    pred_y = neigh.predict(OH_enc.transform(X_val))
    acc = get_accuracy(y_val, pred_y)
    return neigh, acc

def eval_classifier(X, y, one_hot, classifier):
    pred_y = classifier.predict(one_hot.transform(X))
    acc = get_accuracy(y, pred_y)
    return acc

def fit_classifier(X_train, y_train, X_val, y_val, one_hot, classifier_model, **kwargs):
    classifier = classifier_model(**kwargs)
    classifier.fit(one_hot.transform(X_train), y_train)
    acc = eval_classifier(X_val, y_val, one_hot, classifier)
    return classifier, acc

def get_accuracy(y_label, y_pred):
    assert len(y_label) == len(y_pred)
    acc = 100 * sum(y_label == y_pred) / len(y_label)
    return acc

2. Load the datasets into Pandas dataframes or NumPy objects (i.e., arrays or matrices) in Python.

In [3]:
df = load_adult(data_rootdir)

if remove_dup is True:
    df = df.drop_duplicates()

# Get faulty columns in the df - ones with na values or ' :?' values
faulty_cols = get_faulty_col_names(df)
print(faulty_cols)
df.head()

Number of ' ?' in workclass: 1836
Number of ' ?' in occupation: 1843
Number of ' ?' in native-country: 582
['workclass', 'occupation', 'native-country']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Faulty columns can be: 
1. Replaced with mode
2. KNN imputed
3. Retained

### Replace with Mode

In [4]:
clean_train_df, faulty_col_modes = replace_with_mode(df, faulty_cols)
assert get_faulty_col_names(clean_train_df) == []

c:\users\asus\anaconda3\envs\dreamerv2\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [5]:
df_without_QM = df.copy()
for col in faulty_cols:
    df_without_QM = df_without_QM[getattr(df, col) != ' ?']

mapping = {}
for col in faulty_cols:
    mapping[col] = df_without_QM[col].mode().values[0]

clean_train_df = replace_with_mapping(df, faulty_cols, mapping)
assert get_faulty_col_names(clean_train_df) == []
print(len(clean_train_df))

c:\users\asus\anaconda3\envs\dreamerv2\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


32537


## Remove ' ?' in workclass, occupation, native-country

In [6]:
# df = adult_data
# print(len(df))
# for col_name in col_names_with_nan:
#     df = df[getattr(df, col_name) != ' ?']
# print(len(df))

In [7]:
# df.hist(figsize=(16, 16), bins=100)

In [8]:
# df['native-country'].value_counts().index
# # df['native-country'].unique()

In [9]:
# cat_cols = adult_data.select_dtypes(include=['object']).columns.tolist()
# n = len(cat_cols)
# ncols = 3
# nrows = (n // ncols + 1) if n % ncols != 0 else (n // ncols)
# plt.figure(figsize=(16 * nrows, 8 * ncols))
# for idx, col in enumerate(cat_cols):
# #     print(idx)
# #     print(cat_cols)
#     plt.subplot(nrows, ncols, idx  + 1)
#     plt.grid()
#     plt.bar(adult_data[col].value_counts().index, adult_data[col].value_counts().values)
#     plt.title(col)
# plt.tight_layout()
# plt.show()

In [10]:
# cat_cols = df.select_dtypes(include=['object']).columns.tolist()
# n = len(cat_cols)
# ncols = 4
# plt.figure(figsize=(16 * n // ncols, 4 * ncols))
# for idx, col in enumerate(cat_cols):
#     plt.subplot(n // ncols, ncols, idx  + 1)
#     plt.bar( df[col].value_counts().index, df[col].value_counts().values)
#     plt.title(col)
#     plt.grid()
#     # if idx >= 3: break
# plt.tight_layout()
# plt.show()

## Convert Discrete to OneHot

In [11]:
X_train, y_train = clean_train_df.drop('salary', axis=1), get_binary_labels_for_salary(clean_train_df['salary'])

In [12]:
# Get categorical variables
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
print("Discrete Variables: ", cat_cols)

Discrete Variables:  ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


In [13]:
from sklearn.preprocessing import OneHotEncoder
OH_enc = OneHotEncoder(handle_unknown='ignore')
OH_enc.fit(X_train)

OneHotEncoder(handle_unknown='ignore')

In [14]:
test_df = load_adult(data_rootdir, 'test')
print("Before cleaning: ", len(test_df))
faulty_test_cols = get_faulty_col_names(test_df)
print("Columns to clean", faulty_test_cols)

clean_test_df = replace_with_mapping(test_df, faulty_test_cols, mapping)
assert get_faulty_col_names(clean_test_df) == []
print("After cleaning: ", len(clean_test_df))

Before cleaning:  16281
Number of ' ?' in workclass: 963
Number of ' ?' in occupation: 966
Number of ' ?' in native-country: 274
Columns to clean ['workclass', 'occupation', 'native-country']
After cleaning:  16281


In [15]:
X_test, y_test = clean_test_df.drop('salary', axis=1), get_binary_labels_for_salary(clean_test_df['salary'])

## KNN Classifiers

Hyperparameters
- `n_neighbors` : [2:10:2]
- `p`: [1:2:1]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
n_neighbors=3
knn_kw = {
    'n_neighbors': n_neighbors,
}
neigh = KNeighborsClassifier(**knn_kw)
neigh.fit(OH_enc.transform(X_train), y_train)

print("Train Accuracy: ", eval_classifier(X_train, y_train, OH_enc, neigh))
print("Test Accuracy: ", eval_classifier(X_test, y_test, OH_enc, neigh))

In [ ]:
from tqdm.auto import tqdm
n_folds = 5
models = []
accs = []
for batch in tqdm(generate_cross_validation(n_folds, X_train, y_train), total=n_folds):
    cross_X_train, cross_y_train, cross_X_val, cross_y_val = batch
    model, acc = fit_classifier(cross_X_train, cross_y_train, cross_X_val, cross_y_val, OH_enc, KNeighborsClassifier, **knn_kw)
    models.append(model)
    accs.append(acc)

print(accs)
print(f"Model accuracy averaged across {n_folds} folds: {np.mean(accs)}")

## DecisionTreeClassifier

Hyperparameters
- `criterion`: ['gini', 'entropy']
- `max_depth`: [5:20:5]
- `min_samples_split`: [2:6:2]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
criterion='entropy'
max_depth=10
dt_kw = {
    'criterion': criterion,
    'max_depth': max_depth,
}
classifier = DecisionTreeClassifier(**dt_kw)
classifier.fit(OH_enc.transform(X_train), y_train)

print("Train Accuracy: ", eval_classifier(X_train, y_train, OH_enc, classifier))
print("Test Accuracy: ", eval_classifier(X_test, y_test, OH_enc, classifier))

In [ ]:
from tqdm.auto import tqdm
n_folds = 5
models = []
accs = []
for batch in tqdm(generate_cross_validation(n_folds, X_train, y_train), total=n_folds):
    cross_X_train, cross_y_train, cross_X_val, cross_y_val = batch
    model, acc = fit_classifier(cross_X_train, cross_y_train, cross_X_val, cross_y_val, OH_enc, DecisionTreeClassifier, **dt_kw)
    models.append(model)
    accs.append(acc)

print(accs)
print(f"Model accuracy averaged across {n_folds} folds: {np.mean(accs)}")